## Fetch VCF and index

In [13]:
import sgkit
! mkdir -p data
! test ! -e data/chr22.vcf.gz && wget -O data/chr22.vcf.gz http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000G_2504_high_coverage/working/20201028_3202_phased/CCDG_14151_B01_GRM_WGS_2020-08-05_chr22.filtered.shapeit2-duohmm-phased.vcf.gz

In [15]:
! test ! -e data/chr22.vcf.gz.tbi && tabix -f -p vcf data/chr22.vcf.gz

## Fetch ancestral alleles and index

In [16]:
! test ! -e data/ancestral_alleles.tar.gz && wget -O data/ancestral_alleles.tar.gz  ftp://ftp.ensembl.org/pub/release-100/fasta/ancestral_alleles/homo_sapiens_ancestor_GRCh38.tar.gz

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/benj/.wget-hsts'. HSTS will be disabled.
--2023-01-19 00:40:39--  ftp://ftp.ensembl.org/pub/release-100/fasta/ancestral_alleles/homo_sapiens_ancestor_GRCh38.tar.gz
           => ‘data/ancestral_alleles.tar.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-100/fasta/ancestral_alleles ... done.
==> SIZE homo_sapiens_ancestor_GRCh38.tar.gz ... 852605016
==> PASV ... done.    ==> RETR homo_sapiens_ancestor_GRCh38.tar.gz ... done.
Length: 852605016 (813M) (unauthoritative)

homo_sapiens_ancest 100%[===================>] 813.11M  5.30MB/s    in 2m 33s  

2023-01-19 00:43:12 (5.33 MB/s) - ‘data/ancestral_alleles.tar.gz’ saved [852605016]



In [17]:
! cd data && tar -xzf ancestral_alleles.tar.gz

In [18]:
! samtools faidx data/homo_sapiens_ancestor_GRCh38/homo_sapiens_ancestor_22.fa

## Convert VCF to an sgkit dataset

In [4]:
%%time
import sgkit as sg
import xarray as xr
import numpy as np
import dask.array as da
from sgkit.io.vcf import vcf_to_zarr

CPU times: user 1.18 s, sys: 253 ms, total: 1.43 s
Wall time: 2.67 s


In [20]:
%%time
vcf_to_zarr("data/chr22.vcf.gz", "data/chr22.zarr")

CPU times: user 7min 57s, sys: 32.6 s, total: 8min 29s
Wall time: 3min 21s


# Load ancestral states from fasta and save to dataset

In [38]:
import pysam
import sys
fasta = pysam.FastaFile("data/homo_sapiens_ancestor_GRCh38/homo_sapiens_ancestor_22.fa")
# NB! We put in an extra character at the start to convert to 1 based coords.
codec = 'utf-32-le' if sys.byteorder == 'little' else 'utf-32-be'
ancestral_sequence = "X" + fasta.fetch(reference=fasta.references[0])
ancestral_sequence = np.frombuffer(bytearray(ancestral_sequence,codec), dtype="U1")
# From the ancestral states README:
# The convention for the sequence is:
#    ACTG : high-confidence call, ancestral state supported by other 2 sequences
#    actg : low-confidence call, ancestral state supported by one sequence only
#    N    : failure, the ancestral state is not supported by any other sequence
#    -    : the extant species contains an insertion at this postion
#    .    : no coverage in the alignment
ds = sg.load_dataset("data/chr22.zarr")
ancestral_states = ancestral_sequence[ds['variant_position'].values]
ds.update({"variant_ancestral_state": xr.DataArray(data=ancestral_states, dims=["variants"], name="variant_ancestral_state")})
sg.save_dataset(ds.drop_vars(set(ds.data_vars) - {"variant_ancestral_state"}), "data/chr22.zarr", mode="a")

## Remove sites with duplicate positions

In [32]:
ds = sg.load_dataset("data/chr22.zarr")
print(f"{ds.sizes['variants']} sites available")
pos = ds['variant_position']
pos_shift_left = da.full_like(pos,-1)
pos_shift_left[0:-1] = pos[1:]
pos_shift_right = da.full_like(pos,-1)
pos_shift_right[1:] = pos[:-1]
wanted_variants = da.logical_and(pos != pos_shift_left, pos != pos_shift_right)
ds.update({"variant_mask": xr.DataArray(data=wanted_variants, dims=["variants"], name="variant_mask")})
sg.save_dataset(ds.drop_vars(set(ds.data_vars) - {"variant_mask"}), "data/chr22.zarr", mode="a")
print(f"{da.sum(wanted_variants).compute()} sites used")

1070401 sites available
970897 sites used


## Take a subset of the samples for testing

In [41]:
ds = sg.load_dataset("data/chr22.zarr")
wanted_samples = np.zeros((ds.sizes['samples'],), dtype=bool)
wanted_samples[:100] = True
ds = ds.sel(samples=wanted_samples)
sg.save_dataset(ds, "data/chr22.subset.zarr")

/home/benj/projects/ukb/nygc1000_notebook/env/lib/python3.10/site-packages/xarray/conventions.py:205: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/benj/projects/ukb/nygc1000_notebook/env/lib/python3.10/site-packages/xarray/conventions.py:205: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(


In [2]:
%%time
import tsinfer
sampledata = tsinfer.SgkitSampleData("data/chr22.subset.zarr")
inf_ts = tsinfer.infer(sampledata, num_threads=4, progress_monitor=True)

ga-add   (1/6)100%|██████████| 971k/971k [00:22, 42.8kit/s] 
ga-gen   (2/6)100%|██████████| 77.0k/77.0k [06:16, 204it/s]
ma-match (3/6)100%|█████████▉| 77.0k/77.0k [02:33, 408it/s]
ms-muts  (4/6)  0%|          | 0.00/155k [00:00, ?it/s]
ms-muts  (4/6)  0%|          | 1.00/155k [00:00, 2.28it/s]
ms-muts  (4/6)  3%|▎         | 5.18k/155k [00:00, 9.69kit/s]
ms-muts  (4/6)  7%|▋         | 10.7k/155k [00:00, 17.0kit/s]
ms-muts  (4/6) 11%|█         | 16.4k/155k [00:00, 22.5kit/s]
ms-muts  (4/6) 14%|█▍        | 22.3k/155k [00:00, 27.0kit/s]
ms-muts  (4/6) 18%|█▊        | 28.1k/155k [00:00, 30.3kit/s]
ms-muts  (4/6) 22%|██▏       | 34.2k/155k [00:01, 33.5kit/s]
ms-muts  (4/6) 26%|██▌       | 40.5k/155k [00:01, 36.2kit/s]
ms-muts  (4/6) 30%|███       | 46.6k/155k [00:01, 38.3kit/s]
ms-muts  (4/6) 34%|███▍      | 52.8k/155k [00:01, 40.1kit/s]
ms-muts  (4/6) 38%|███▊      | 59.0k/155k [00:01, 41.8kit/s]
ms-muts  (4/6) 42%|████▏     | 65.3k/155k [00:01, 43.2kit/s]
ms-muts  (4/6) 46%|████▌     | 71

CPU times: user 19min 21s, sys: 10.8 s, total: 19min 32s
Wall time: 10min 45s


In [3]:
inf_ts.num_sites

970897

In [14]:
inf_ts